In [37]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import os, shutil
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
from PIL import ImageOps
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


data_dir="/Users/huangjie/ee541_super_resulotion/data"

# Load the data using the ImageFolder dataset class
batch_size=32

train_hr = os.path.join(data_dir,'training_hr')
train_lr = os.path.join(data_dir,'training_lr/X2')
val_hr = os.path.join(data_dir,'validation_hr')
val_lr = os.path.join(data_dir,'validation_lr/X2')
test_hr = os.path.join(data_dir,'test_hr')
test_lr = os.path.join(data_dir,'test_lr/X2')


In [38]:
#found from before
min_height = 24
min_width = 24

# Define the transformations to be applied to the images
seed = 123
generator = torch.Generator()
generator.manual_seed(seed)

transform = {
    'train_input': transforms.Compose([
        transforms.CenterCrop([min_height,min_width]),
        # transforms.RandomApply([
        #   transforms.RandomHorizontalFlip(p=0.2),
        #   transforms.RandomVerticalFlip(p=0.2),
        #   transforms.RandomRotation(30),
        # ], p=0.5),
        transforms.ToTensor(),
    ]),
    'train_output': transforms.Compose([
        transforms.CenterCrop([min_height*2,min_width*2]),
        # transforms.RandomApply([
        #   transforms.RandomHorizontalFlip(p=0.2),
        #   transforms.RandomVerticalFlip(p=0.2),
        #   transforms.RandomRotation(30),
        # ], p=0.5),
        transforms.ToTensor(),
    ]),
    'val_test_input': transforms.Compose([
        transforms.CenterCrop([min_height,min_width]),
        transforms.ToTensor(),
    ]),
    'val_test_output': transforms.Compose([
        transforms.CenterCrop([min_height*2,min_width*2]),
        transforms.ToTensor(),
    ])
}

In [39]:
class Div2K(Dataset):
  def __init__(self, input_folder, output_folder, transform_input, transform_output):
    self.input_folder = input_folder
    self.output_folder = output_folder
    self.transform_input = transform_input
    self.transform_output = transform_output
    self.input_images = sorted([file for file in os.listdir(input_folder) if file.endswith('.png')])
    self.output_images = sorted([file for file in os.listdir(output_folder) if file.endswith('.png')])
    
  def __len__(self):
    return len(self.input_images)
  def __getitem__(self,index):
    input_image_path = os.path.join(self.input_folder, self.input_images[index])
    output_image_path = os.path.join(self.output_folder, self.output_images[index])
    input_image = Image.open(input_image_path).convert('RGB')
    output_image = Image.open(output_image_path).convert('RGB')

    input_image = self.transform_input(input_image)
    output_image = self.transform_output(output_image)
    return input_image, output_image

train_dataset = Div2K(train_lr, train_hr, transform_input=transform['train_input'],transform_output=transform['train_output'])
test_dataset = Div2K(test_lr, test_hr, transform_input = transform['val_test_input'],transform_output=transform['val_test_output'])
val_dataset = Div2K(val_lr, val_hr, transform_input = transform['val_test_input'],transform_output=transform['val_test_output'])

batch_size = 3

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_dataloader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

i=0
for lr_images,hr_images in train_dataloader:
    # Plot the first image in the batch
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    #expected shape (height, width, num_channels)
    ax[0].imshow(lr_images[0])
    ax[0].set_title('Low-resolution')
    ax[1].imshow(hr_images[0])
    ax[1].set_title('High-resolution')
    plt.show()
    i+=1
    if(i>2):
        break

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)    exitcode = _main(fd, parent_sentinel)

  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
    self = reduction.pickle.load(from_parent)
AttributeError: AttributeErrorCan't get attribute 'Div2K' on <module '__main__' (built-in)>
: Can't get attribute 'Div2K' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 85772, 85773) exited unexpectedly

In [22]:
class SRResNet(nn.Module):
    def __init__(self, upscale_factor=4, num_channels=64, num_residual_blocks=16):
        super(SRResNet, self).__init__()

        # First convolutional layer
        self.conv1 = nn.Conv2d(3, num_channels, kernel_size=9, padding=4)

        # Residual blocks
        res_blocks = []
        for _ in range(num_residual_blocks):
            res_blocks.append(ResidualBlock(num_channels))

        self.res_blocks = nn.Sequential(*res_blocks)

        # Second convolutional layer
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)

        # Upscaling layers
        upsample_blocks = []
        for _ in range(int(upscale_factor/2)):
            upsample_blocks.append(UpsampleBlock(num_channels))

        self.upsample = nn.Sequential(*upsample_blocks)

        # Third convolutional layer
        self.conv3 = nn.Conv2d(num_channels, 3, kernel_size=9, padding=4)

    def forward(self, x):
        out = self.conv1(x)
        out = self.res_blocks(out)
        out = self.conv2(out)
        out = self.upsample(out)
        out = self.conv3(out)
        return out

class ResidualBlock(nn.Module):
    def __init__(self, num_channels):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        return out

class UpsampleBlock(nn.Module):
    def __init__(self, num_channels):
        super(UpsampleBlock, self).__init__()

        self.conv = nn.Conv2d(num_channels, 4*num_channels, kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(2)
        self.prelu = nn.PReLU()

    def forward(self, x):
        out = self.conv(x)
        out = self.pixel_shuffle(out)
        out = self.prelu(out)
        return out



In [31]:
#Chekcihng dataloader dimensions (opt)
data_iter = iter(train_dataloader)
lr_images, hr_images = next(data_iter)
print("Low-resolution images:")
print("Shape:", lr_images.shape)
print("Data type:", lr_images.dtype)

print("\nHigh-resolution images:")
print("Shape:", hr_images.shape)
print("Data type:", hr_images.dtype)

#displaying some image pairs (opt)
i=0
for lr_images,hr_images in train_dataloader:
    # Plot the first image in the batch
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    #expected shape (height, width, num_channels)
    lr_image = lr_images[0].permute(1, 2, 0)
    ax[0].imshow(lr_image)
    ax[0].set_title('Low-resolution')
    hr_image = hr_images[0].permute(1, 2, 0)
    ax[1].imshow(hr_image)
    ax[1].set_title('High-resolution')
    plt.show()
    i+=1
    if(i>2):
        break

Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'Div2K' on <module '__main__' (built-in)>
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'Div2K' on <module '__main__' (built-in)>


RuntimeError: DataLoader worker (pid(s) 85611, 85612) exited unexpectedly

In [23]:
import torch.optim as optim
torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SRResNet()
model.to(device)


criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
for epoch in range(10):
    train_loss = 0.0
    val_loss = 0.0
    model.train()
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_dataloader.dataset)

    model.eval()
    with torch.no_grad():
        for inputs, targets in val_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
        val_loss /= len(val_dataloader.dataset)

    print('Epoch: [{}/{}]\tTraining Loss: {:.4f}\tValidation Loss: {:.4f}'.format(epoch+1, 10, train_loss, val_loss))


Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
  File "<string>", line 1, in <module>
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/huangjie/opt/anaconda3/envs/ee541_work/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
    self = reduction.pickle.load(from_parent)
AttributeErrorAttributeError: : Can't get attribute 'Div2K' on <module '__main__' (built-in)>Can't get attribute 'Div2K' on <module '__main__' (built-in)>



RuntimeError: DataLoader worker (pid(s) 85571, 85572) exited unexpectedly